In [ ]:
#<----- Click this play button to start MocapNET setup!
#If you deploy this on your PC, run : jupyter notebook initialize.ipynb
!git clone -b mnet4 https://github.com/FORTH-ModelBasedTracker/MocapNET.git
!MocapNET/src/python/mnet4/setup.sh --collab
import os
os.chdir("/content/MocapNET/src/python/mnet4")
print("MocapNET setup is finished, you can run the next cell now..")


In [ ]:
#Get an interesting video from the internet, to track ( e.g. shuffle.webm )
#You can also upload your own by clicking the Files icon and using the menu on the left
#You need to put files in the /content/MocapNET/src/python/mnet4/ directory
!wget http://ammar.gr/mocapnet/shuffle.webm

#Analyze the file shuffle.web through MediaPipe 2D + MocapNET 3D Pose Estimation!
!python3 -m mediapipeHolisticWebcamMocapNET --from shuffle.webm --ik 0.001 99 99 --all --save --plot --headless
print("MocapNET video input processing finished, you can run the next cells now to see the results..")

#Outputs are :
#livelastRun3DHiRes.mp4 | A video showing the regressed output overlayed on RGB
#livelastPlot3DHiRes.mp4| A video plot of the retrieved BVH degrees of freedom
#out.bvh                | the extracted BVH file 
#2d_out.csv             | Input 2D joints used by MocapNET as a CSV file
#3d_out.csv             | Output 3D joints produced by MocapNET as a CSV file
#bvh_out.csv            | Output BVH angles produced by MocapNET as a CSV file

#To render the BVH file see this Video: 
#            https://youtu.be/ooLRUS5j4AI

In [ ]:
#<- Run a sign-language dataset without mediapipe from a dumped Openpose/JSON/CSV source
!wget http://ammar.gr/datasets/signumtest.zip -O signumtest.zip
!unzip -o signumtest.zip

#Analyze the file shuffle.web through MediaPipe 2D + MocapNET 3D Pose Estimation!
!python3 -m csvNET --from con0014/2dJoints_v1.4.csv --ik 0.001 99 99 --all --save --plot --headless
print("MocapNET video input processing finished, you can run the next cells now to see the results..")

In [ ]:
#Show generated BVH angle plots!
from IPython.display import Video
Video("livelastPlot3DHiRes.mp4",embed=True)

In [ ]:
#Show generated Video File
from IPython.display import Video
Video("livelastRun3DHiRes.mp4",embed=True)

In [ ]:
from google.colab import files

# Trigger the file download
print("Download Output Files!")
files.download("out.bvh")files.download("2d_out.csv")files.download("3d_out.csv")files.download("bvh_out.csv")

In [2]:
%%HTML
<iframe width="560" height="315" src="https://youtube.com/embed/ooLRUS5j4AI"></iframe>